In [180]:
# #!/bin/bash
# !curl -L -o ./credit-score-classification.zip\
#   https://www.kaggle.com/api/v1/datasets/download/parisrohan/credit-score-classification

In [181]:
import pandas as pd
from typing import Optional
from IPython.utils import io
from IPython.display import display

In [182]:
file_name = "train_raw.csv"
df = pd.read_csv(file_name)

/tmp/ipykernel_30383/4186681340.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


In [183]:
column_idx_with_issue = 26
column_name = df.columns[column_idx_with_issue]
column_name

'Monthly_Balance'

In [184]:
df.dtypes

ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                          object
SSN                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
Credit_Score                

In [185]:
# create boolean mask of rows where the value is a Python string
mask = df[column_name].map(lambda x: isinstance(x, str))
df.loc[mask, column_name].head(5)

0    312.49408867943663
1    284.62916249607184
2     331.2098628537912
3    223.45130972736786
4    341.48923103222177
Name: Monthly_Balance, dtype: object

Find rows that cannot be parsed as numeric (useful if column should be numeric):

In [186]:
# re-read that column as strings to see raw values
raw = pd.read_csv(file_name, dtype=str, usecols=[column_name])
raw_vals = raw[column_name].dropna().head(200)
raw_vals.head(50)
# find values containing non-digit characters (excluding dot/minus)
import re
pattern = r'[^0-9\.\-]'
raw.loc[raw[column_name].str.contains(pattern, na=False), column_name].value_counts().head(50)

Monthly_Balance
__-333333333333333333333333333__    9
Name: count, dtype: int64

In [187]:
# value you found
bad_val = "__-333333333333333333333333333__"

# find how many and which row indices
mask = df[column_name] == bad_val
print("count:", mask.sum())
print("indices:", df.loc[mask].index.tolist())

# show the full rows for context
df.loc[mask].head(10)

count: 9
indices: [5545, 26177, 29158, 35570, 38622, 60009, 75251, 82918, 83255]


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
5545,0x367f,CUS_0x9885,February,Paul Dayl,17,186-49-9679,Developer,41086.4,3660.866667,3,...,Standard,113.98,34.826541,11 Years and 6 Months,Yes,145.544320,201.1429928919469,Low_spent_Medium_value_payments,__-333333333333333333333333333__,Standard
26177,0xaf63,CUS_0x5a90,February,Scuffhamq,45,264-53-1943,Engineer,29728.31,2551.359167,3,...,Good,917.0,38.198076,31 Years and 9 Months,No,34.496445,198.6115126951797,Low_spent_Large_value_payments,__-333333333333333333333333333__,Good
29158,0xc0d8,CUS_0x288d,July,Selamk,55,970-19-8342,Lawyer,30748.93,2572.410833,1,...,Good,860.97,40.634250,19 Years and 5 Months,No,43.727774,79.07180674947499,High_spent_Small_value_payments,__-333333333333333333333333333__,Standard
35570,0xe66c,CUS_0x85e9,March,Anna Driveri,45,288-62-7562,_______,20929.295_,1704.107917,5,...,_,858.09,33.943626,21 Years and 7 Months,No,13798.000000,220.30455197823662,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
38622,0xf84c,CUS_0x2b77,July,Longstretho,21,006-36-0111,Doctor,15167.62,1471.968333,4,...,Good,847.03,28.541848,NaN,NM,9.930809,128.87587207680795,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
60009,0x1759f,CUS_0xc06e,February,radenp,30,046-72-5387,Journalist,92114.68,7811.223333,3,...,Standard,820.52,39.089475,20 Years and 10 Months,Yes,54.219664,NaN,High_spent_Large_value_payments,__-333333333333333333333333333__,Standard
75251,0x1ceed,CUS_0x57f3,April,Kristen Haysq,29,577-85-0830,Mechanic,60904.59,NaN,3,...,Standard,2441.47,29.878716,12 Years and 11 Months,Yes,61.525140,394.8939566733383,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
82918,0x1fbd8,CUS_0x41bf,July,Doeringq,44,693-64-7611,Journalist,61990.52,4900.876667,10,...,Bad,3618.91,37.718591,13 Years and 8 Months,NM,355.143396,194.28348839148083,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
83255,0x1fdd1,CUS_0x2f7e,August,Sabina Zawadzkif,31,109-48-2095,Doctor,126353.04,10635.420000,4,...,_,859.77,28.442867,31 Years and 9 Months,No,167.043910,549.7661392945339,Low_spent_Medium_value_payments,__-333333333333333333333333333__,Poor


In [188]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [189]:
import pandas as pd

# Example schema
dtypes = {
    "Customer_ID": "string",
    "Month": "string",
    "Name": "string",
    "Age": "float64",
    "SSN": "string",
    "Occupation": "string",
    "Annual_Income": "float64",
    "Monthly_Inhand_Salary": "float64",
    "Num_Bank_Accounts": "int",
    "Num_Credit_Card": "int",
    "Interest_Rate": "float64",
    "Num_of_Loan": "int",
    "Type_of_Loan": "string",
    "Delay_from_due_date": "int",
    "Num_of_Delayed_Payment": "int",
    "Changed_Credit_Limit": "float64",
    "Num_Credit_Inquiries": "float64",
    "Credit_Mix": "string",
    "Outstanding_Debt": "float64",
    "Credit_Utilization_Ratio": "float64",
    "Credit_History_Age": "string",
    "Payment_of_Min_Amount": "string",
    "Total_EMI_per_month": "float64",
    "Amount_invested_monthly": "float64",
    "Payment_Behaviour": "string",
    "Monthly_Balance": "float64",
    "Credit_Score": "string"
}

# Read raw CSV
df = pd.read_csv(file_name)

# Apply schema with coercion
for col, typ in dtypes.items():
    if typ.startswith("float") or typ.startswith("int"):
        df[col] = pd.to_numeric(df[col], errors="coerce")  # invalid -> NaN
    elif typ == "string":
        df[col] = df[col].astype("string").replace(r"^\s*$", pd.NA, regex=True)


/tmp/ipykernel_30383/3223048264.py:35: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


In [190]:
df[::40005].T

,0,40005,80010
ID,0x1602,0x10067,0x1ead0
Customer_ID,CUS_0xd40,CUS_0x41be,CUS_0x7d5e
Month,January,June,March
Name,Aaron Maashoh,Selyukha,<NA>
Age,23.0,35.0,19.0
SSN,821-00-0265,700-01-5042,063-39-1383
Occupation,Scientist,Musician,Journalist
Annual_Income,19114.12,NaN,NaN
Monthly_Inhand_Salary,1824.843333,6093.61,2409.553333
Num_Bank_Accounts,3,7,6


In [191]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  string 
 2   Month                     100000 non-null  string 
 3   Name                      90015 non-null   string 
 4   Age                       95061 non-null   float64
 5   SSN                       100000 non-null  string 
 6   Occupation                100000 non-null  string 
 7   Annual_Income             93020 non-null   float64
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               95215 non-null   float64
 13  Type_of_Loan              88592 non-null   st

In [192]:
df[::10000]

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.120,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good
10000,0x509a,CUS_0x229f,January,all Davidt,45.0,547-53-2472,_______,20261.850,1598.487500,7,...,Bad,1300.71,34.779107,14 Years and 2 Months,Yes,96.409635,124.514960,Low_spent_Small_value_payments,228.924155,Poor
20000,0x8b32,CUS_0x10c0,January,Scheyderx,38.0,658-53-1756,Architect,49454.130,4328.177500,8,...,Bad,1388.56,34.322164,12 Years and 9 Months,Yes,73.346359,304.051202,!@9#%8,345.420189,Standard
30000,0xc5ca,CUS_0x236b,January,Martinneo,25.0,575-63-7039,Manager,18393.510,1742.792500,9,...,_,2604.09,29.119227,4 Years and 3 Months,Yes,62.204931,233.032353,Low_spent_Small_value_payments,169.041966,Standard
40000,0x10062,CUS_0x41be,January,Selyukha,35.0,700-01-5042,Musician,72583.320,6093.610000,7,...,Standard,1019.44,37.220639,18 Years and 2 Months,Yes,104.016730,334.484535,Low_spent_Medium_value_payments,450.859735,Standard
50000,0x13afa,CUS_0x2646,January,ewich Billk,45.0,261-90-8736,Journalist,7807.285,606.607083,4,...,Standard,692.63,26.650258,<NA>,Yes,7.771075,31.082811,Low_spent_Small_value_payments,311.806822,Standard
60000,0x17592,CUS_0xa505,January,Jonathan Stempelz,41.0,959-35-8150,Engineer,34899.220,2764.268333,10,...,Standard,2103.76,30.962745,7 Years and 8 Months,Yes,78.967677,110.466661,Low_spent_Small_value_payments,376.992495,Poor
70000,0x1b02a,CUS_0x1d1b,January,Araxiac,20.0,#F%$D@*&8,Manager,152574.760,12547.563333,2,...,Good,NaN,38.632872,20 Years and 1 Months,No,205.484843,974.560691,Low_spent_Small_value_payments,364.710799,Standard
80000,0x1eac2,CUS_0x9df0,January,Timk,1642.0,846-31-4164,Entrepreneur,36115.920,2716.660000,0,...,_,886.95,27.797266,22 Years and 9 Months,No,78.052255,96.001442,High_spent_Small_value_payments,357.612303,Good
90000,0x2255a,CUS_0x3787,January,Kajimotoz,52.0,986-71-6476,Engineer,21270.790,1485.565833,6,...,Standard,782.15,39.263213,16 Years and 9 Months,Yes,48.481182,63.795843,High_spent_Small_value_payments,296.279558,Standard


### Convert column names to lower case

In [193]:
df.columns = df.columns.str.lower()

### Display unique values to analyze data

In [194]:
from IPython.display import display
def display_column_unique_values():
    for col in df.columns:
        print(f"Column Name: {col}, Unique count: {df[col].nunique()}")
        print(df[col].unique())
        print("-" * 20)

In [195]:
with io.capture_output() as captured:
    display_column_unique_values()

captured_output_text = captured.stdout
captured_output_text

"Column Name: id, Unique count: 100000\n['0x1602' '0x1603' '0x1604' ... '0x25feb' '0x25fec' '0x25fed']\n--------------------\nColumn Name: customer_id, Unique count: 12500\n<StringArray>\n[ 'CUS_0xd40', 'CUS_0x21b1', 'CUS_0x2dbc', 'CUS_0xb891', 'CUS_0x1cdb',\n 'CUS_0x95ee', 'CUS_0x284a', 'CUS_0x5407', 'CUS_0x4157', 'CUS_0xba08',\n ...\n 'CUS_0x2654', 'CUS_0xb11c', 'CUS_0xad4f', 'CUS_0x51b3', 'CUS_0x2084',\n 'CUS_0x372c',  'CUS_0xf16', 'CUS_0xaf61', 'CUS_0x8600', 'CUS_0x942c']\nLength: 12500, dtype: string\n--------------------\nColumn Name: month, Unique count: 8\n<StringArray>\n['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August']\nLength: 8, dtype: string\n--------------------\nColumn Name: name, Unique count: 10139\n<StringArray>\n[   'Aaron Maashoh',               <NA>,  'Rick Rothackerj',\n           'Langep',           'Jasond',           'Deepaa',\n               'Np',           'Nadiaq',             'Annk',\n     'Charlie Zhur',\n ...\n  'Rick Rothackeru', 

## Replace all incorrect values with `pd.na`

### Data Quality Note: SSN Column
- **Total rows:** 100,000
- **Invalid SSN values removed:** 5,572 (5.5%)
- **Pattern enforced:** `\d{3}-\d{2}-\d{4}`
- **Action:** Replaced invalid entries with `pd.NA`
- **Justification:** SSN is an identifier, not a predictive feature. Invalid tokens (e.g., `#F%$D@*&8`) indicate data corruption.

In [196]:
df["ssn"] = df["ssn"].where(df["ssn"].str.match(r"^\d{3}-\d{2}-\d{4}$", na=False), pd.NA)

### Implement one-hot encoding for the column `type_of_loan`

#### Analyze a number of unique values in the `type_of_loan` column

In [197]:
df["type_of_loan"].hasnans

True

In [198]:
loan_types = set()

for val in df["type_of_loan"]:
    if pd.isna(val):
        continue

    normalized = val.replace(" and", ", ")
    types = normalized.split(",")
    for type in types:
        if type.strip() == "":
            continue
        loan_types.add(type.strip())

print(loan_types)
print(len(loan_types))

{'Payday Loan', 'Not Specified', 'Student Loan', 'Credit-Builder Loan', 'Debt Consolidation Loan', 'Personal Loan', 'Auto Loan', 'Mortgage Loan', 'Home Equity Loan'}
9


#### Apply one-hot encoding

In [199]:
def apply_one_hot_encoding_to_type_of_loan():
    if "type_of_loan" not in df.columns:
        print("`type_of_loan` has already been removed")
        return

    for loan in sorted(loan_types):
        column_name = f"has_{loan.lower().replace(" ", "_").replace("-", "_")}"
        df[column_name] = df["type_of_loan"].apply(
            lambda s: 1 if pd.notna(s) and loan in s else 0
        ).astype("int8")

In [200]:
apply_one_hot_encoding_to_type_of_loan()

In [201]:
df = df.drop(["type_of_loan"], axis=1, errors="ignore")

In [202]:
df.head()

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,...,credit_score,has_auto_loan,has_credit_builder_loan,has_debt_consolidation_loan,has_home_equity_loan,has_mortgage_loan,has_not_specified,has_payday_loan,has_personal_loan,has_student_loan
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,0,1,0,0,0,1,0
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,1,0,0,0,1,0
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500.0,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,1,0,0,0,1,0
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,1,0,0,0,1,0
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,0,1,0,0,0,1,0


### Cleanup the column `occupation`

In [203]:
df["occupation"] = df["occupation"].replace("_______", pd.NA)
df["occupation"].unique()

<StringArray>
[    'Scientist',            <NA>,       'Teacher',      'Engineer',
  'Entrepreneur',     'Developer',        'Lawyer', 'Media_Manager',
        'Doctor',    'Journalist',       'Manager',    'Accountant',
      'Musician',      'Mechanic',        'Writer',     'Architect']
Length: 16, dtype: string

### Cleanup the column `credit_mix`

In [204]:
df["credit_mix"] = df["credit_mix"].replace("_", pd.NA)

In [205]:
df["credit_mix"].unique()

<StringArray>
[<NA>, 'Good', 'Standard', 'Bad']
Length: 4, dtype: string

### Data Transformation Note: `Credit_History_Age` Column

**Original format:** `"X Years and Y Months"` (string)
- Example: `"22 Years and 1 Months"`, `"0 Years and 11 Months"`
- Total unique values: 404
- Data type: String (not suitable for modeling)

**Problem:**
- Non-numeric format prevents mathematical operations
- Cannot be used directly in ML models
- Inconsistent scale (years vs months)

**Solution:** Convert to numeric total months

**Transformation applied:**
```
"X Years and Y Months" → (X × 12) + Y months
```

**Examples:**
- `"22 Years and 1 Months"` → 265 months
- `"0 Years and 11 Months"` → 11 months
- `"1 Years and 0 Months"` → 12 months

**New column:** `credit_history_months` (Int32, nullable)
- Preserves ordering and scale
- Suitable for distance-based and linear models
- Missing values retained as `<NA>`

In [206]:
def convert_string_to_months(str: str) -> Optional[int]:
    if pd.isna(str):
        return pd.NA

    values = str.split(" and ")
    years = int(values[0].split(" Years")[0])
    months = int(values[1].split(" Months")[0])

    return years * 12 + months

In [207]:
df["credit_history_months"] = df["credit_history_age"].apply(
    lambda s: convert_string_to_months(s)
)

In [208]:
df = df.drop(["credit_history_age"], axis=1, errors="ignore")

In [209]:
df.head()

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,...,has_auto_loan,has_credit_builder_loan,has_debt_consolidation_loan,has_home_equity_loan,has_mortgage_loan,has_not_specified,has_payday_loan,has_personal_loan,has_student_loan,credit_history_months
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,1824.843333,3,...,1,1,0,1,0,0,0,1,0,265
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,NaN,3,...,1,1,0,1,0,0,0,1,0,<NA>
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500.0,821-00-0265,Scientist,19114.12,NaN,3,...,1,1,0,1,0,0,0,1,0,267
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,NaN,3,...,1,1,0,1,0,0,0,1,0,268
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23.0,821-00-0265,Scientist,19114.12,1824.843333,3,...,1,1,0,1,0,0,0,1,0,269


### Cleaning up the column `payment_of_min_amount`

In [210]:
df["payment_of_min_amount"] = df["payment_of_min_amount"].replace("NM", pd.NA)
df["payment_of_min_amount"].unique()

<StringArray>
['No', <NA>, 'Yes']
Length: 3, dtype: string

### Data Cleaning Note: `payment_behaviour` Column

**Original values:** 7 unique categories
- Valid categories (6): Follow pattern `{Spending_Level}_{Value_Size}_payments`
  - `High_spent_Small_value_payments`
  - `High_spent_Medium_value_payments`
  - `High_spent_Large_value_payments`
  - `Low_spent_Small_value_payments`
  - `Low_spent_Medium_value_payments`
  - `Low_spent_Large_value_payments`
- Invalid value (1): `'!@9#%8'` (7.6% of records)

**Pattern interpretation:**

The `{Spending_Level}_{Value_Size}_payments` format captures two behavioral dimensions:

1. **`Spending_Level`** (High/Low): Total amount spent across all transactions
2. **`Value_Size`** (Small/Medium/Large): Average size of individual transactions

**Examples:**
- `High_spent_Small_value_payments`: Many small transactions totaling high spend (e.g., 100 × $50 = $5,000)
- `Low_spent_Large_value_payments`: Few large transactions totaling low spend (e.g., 2 × $500 = $1,000)

**Problem:**
- `'!@9#%8'` is a corrupted placeholder value (similar to `'_______'`, `'_'`, `'#F%$D@*&8'` seen in other columns)
- Represents 7,600 rows with missing payment behavior data
- Not a valid behavioral category

**Action:** Replace `'!@9#%8'` with `pd.NA`

**Justification:**
- Invalid token indicates data corruption, not a meaningful category
- Consistent with data quality cleanup approach for other columns
- Missing values can be handled during model preprocessing (imputation or dedicated handling)

**Credit risk insight:**
This feature provides valuable behavioral signals:
- High frequency small payments may indicate stable, regular income
- Large infrequent payments may suggest different financial management styles
- Captures spending patterns beyond simple transaction amounts

In [211]:
(df["payment_behaviour"] == "!@9#%8").sum() / len(df) * 100

np.float64(7.6)

In [212]:
df["payment_behaviour"] = df["payment_behaviour"].replace("!@9#%8", pd.NA)

In [213]:
payment_behavior_values = set()

for val in df["payment_behaviour"]:
    if pd.isna(val) is True:
        continue

    parts = val.lower().split("_payments")
    payment_behavior_values.add(parts[0])

payment_behavior_values

{'high_spent_large_value',
 'high_spent_medium_value',
 'high_spent_small_value',
 'low_spent_large_value',
 'low_spent_medium_value',
 'low_spent_small_value'}

In [214]:
for payment_behavior_value in payment_behavior_values:
    column_name = f"payment_behavior_{payment_behavior_value}"

    df[column_name] = df["payment_behaviour"].apply(
        lambda value: 1 if pd.notna(value) and payment_behavior_value in value.lower() else 0
    )

In [215]:
df = df.drop(["payment_behaviour"], axis=1, errors="ignore")

In [216]:
df.head(1).T

,0
id,0x1602
customer_id,CUS_0xd40
month,January
name,Aaron Maashoh
age,23.0
ssn,821-00-0265
occupation,Scientist
annual_income,19114.12
monthly_inhand_salary,1824.843333
num_bank_accounts,3


In [217]:
df[df["num_of_delayed_payment"] < 0]["num_of_delayed_payment"].value_counts()

num_of_delayed_payment
-1.0    308
-2.0    223
-3.0     91
Name: count, dtype: int64

In [218]:
df[df["num_of_delayed_payment"] == -1].head(2).T

,10,698
id,0x1610,0x1a18
customer_id,CUS_0x21b1,CUS_0x71e0
month,March,March
name,Rick Rothackerj,Aruna Viswanathau
age,28.0,28.0
ssn,004-07-5839,330-96-0638
occupation,Teacher,Mechanic
annual_income,NaN,15398.95
monthly_inhand_salary,3037.986667,NaN
num_bank_accounts,2,1


### Data Cleaning Note: `num_of_delayed_payment` Column

**Problem:**
- Found negative values: `-1` (308 rows), `-2` (223 rows), `-3` (91 rows)
- Total: 622 rows (0.6% of data)
- Semantically invalid: Cannot have negative count of delayed payments

**Action:** Replace negative values with `pd.NA`

**Justification:**
- Negative values are placeholder indicators for missing data
- Consistent with cleanup approach for other columns
- Valid range: 0 or positive integers only

In [219]:
df.loc[df["num_of_delayed_payment"] < 0, "num_of_delayed_payment"] = pd.NA
df["num_of_delayed_payment"].unique()

array([7.000e+00,       nan, 4.000e+00, 6.000e+00, 1.000e+00, 0.000e+00,
       8.000e+00, 5.000e+00, 3.000e+00, 9.000e+00, 1.200e+01, 1.500e+01,
       1.700e+01, 1.000e+01, 2.000e+00, 1.100e+01, 1.400e+01, 2.000e+01,
       2.200e+01, 1.300e+01, 1.600e+01, 1.800e+01, 1.900e+01, 2.300e+01,
       2.400e+01, 2.100e+01, 3.318e+03, 3.083e+03, 1.338e+03, 2.600e+01,
       3.104e+03, 2.500e+01, 1.106e+03, 8.340e+02, 2.672e+03, 2.008e+03,
       5.380e+02, 2.700e+01, 3.478e+03, 2.420e+03, 7.070e+02, 7.080e+02,
       3.815e+03, 2.800e+01, 1.867e+03, 2.250e+03, 1.463e+03, 4.126e+03,
       2.882e+03, 1.941e+03, 2.655e+03, 2.628e+03, 1.320e+02, 3.069e+03,
       3.060e+02, 3.539e+03, 3.684e+03, 1.823e+03, 4.128e+03, 1.946e+03,
       8.270e+02, 2.297e+03, 2.566e+03, 9.040e+02, 1.820e+02, 9.290e+02,
       3.568e+03, 2.503e+03, 1.552e+03, 2.812e+03, 1.697e+03, 3.764e+03,
       8.510e+02, 3.905e+03, 9.230e+02, 8.800e+01, 1.668e+03, 3.253e+03,
       8.080e+02, 2.689e+03, 3.858e+03, 6.420e+02, 

In [220]:
df.dtypes

id                                                  object
customer_id                                 string[python]
month                                       string[python]
name                                        string[python]
age                                                float64
ssn                                         string[python]
occupation                                  string[python]
annual_income                                      float64
monthly_inhand_salary                              float64
num_bank_accounts                                    int64
num_credit_card                                      int64
interest_rate                                        int64
num_of_loan                                        float64
delay_from_due_date                                  int64
num_of_delayed_payment                             float64
changed_credit_limit                               float64
num_credit_inquiries                               float

### Data Cleaning Note: `age` Column

**Problem:**
- Found negative values.
- Semantically invalid: Should be in the range between 18 and 100 (including)

**Action:**
- Replace negative values with `pd.NA`
- Replace values more than 100 with `pd.NA`

In [221]:
df["age"] =df["age"].clip(18, 70)
df.loc[(df["age"] < 18) | (df["age"] > 70), "age"]

Series([], Name: age, dtype: float64)

In [222]:
with io.capture_output() as captured:
    display_column_unique_values()

captured_output_text = captured.stdout
captured_output_text

"Column Name: id, Unique count: 100000\n['0x1602' '0x1603' '0x1604' ... '0x25feb' '0x25fec' '0x25fed']\n--------------------\nColumn Name: customer_id, Unique count: 12500\n<StringArray>\n[ 'CUS_0xd40', 'CUS_0x21b1', 'CUS_0x2dbc', 'CUS_0xb891', 'CUS_0x1cdb',\n 'CUS_0x95ee', 'CUS_0x284a', 'CUS_0x5407', 'CUS_0x4157', 'CUS_0xba08',\n ...\n 'CUS_0x2654', 'CUS_0xb11c', 'CUS_0xad4f', 'CUS_0x51b3', 'CUS_0x2084',\n 'CUS_0x372c',  'CUS_0xf16', 'CUS_0xaf61', 'CUS_0x8600', 'CUS_0x942c']\nLength: 12500, dtype: string\n--------------------\nColumn Name: month, Unique count: 8\n<StringArray>\n['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August']\nLength: 8, dtype: string\n--------------------\nColumn Name: name, Unique count: 10139\n<StringArray>\n[   'Aaron Maashoh',               <NA>,  'Rick Rothackerj',\n           'Langep',           'Jasond',           'Deepaa',\n               'Np',           'Nadiaq',             'Annk',\n     'Charlie Zhur',\n ...\n  'Rick Rothackeru', 

### Replace impossible values (negative, huge) for other columns

In [223]:
for col in ['num_bank_accounts', 'num_credit_card', 'num_of_loan', 'interest_rate']:
    df.loc[(df[col] < 0) | (df[col] > 50), col] = pd.NA

In [225]:
num_cols = df.select_dtypes(include=["number"]).columns
num_cols

Index(['age', 'annual_income', 'monthly_inhand_salary', 'num_bank_accounts',
       'num_credit_card', 'interest_rate', 'num_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'outstanding_debt', 'credit_utilization_ratio',
       'total_emi_per_month', 'amount_invested_monthly', 'monthly_balance',
       'has_auto_loan', 'has_credit_builder_loan',
       'has_debt_consolidation_loan', 'has_home_equity_loan',
       'has_mortgage_loan', 'has_not_specified', 'has_payday_loan',
       'has_personal_loan', 'has_student_loan',
       'payment_behavior_high_spent_large_value',
       'payment_behavior_low_spent_large_value',
       'payment_behavior_high_spent_medium_value',
       'payment_behavior_low_spent_small_value',
       'payment_behavior_high_spent_small_value',
       'payment_behavior_low_spent_medium_value'],
      dtype='object')

In [227]:
df.to_csv("train.csv", index=False)